In [312]:
import pandas as pd

JPG_PATH = "data/csv_files/jpg_file_paths.csv"
META_DATA = "data/csv_files/mimic-cxr-2.0.0-metadata.csv"
LABEL = "data/csv_files/mimic-cxr-2.0.0-chexpert.csv"

In [314]:
jpg_paths = (pd.read_csv(JPG_PATH,
                            usecols=['dicom_id', 'path']
                            )[['dicom_id', 'path']]).set_index(['dicom_id']).sort_values(by='dicom_id')
print(f"df_jpg_paths shape: {jpg_paths.shape}")
jpg_paths.head(10)

df_jpg_paths shape: (377110, 1)


,path
dicom_id,
00000218-9fb20d4e-86045713-8013e08b-0d5bebba,/data8/jpg/files/p17/p17191670/s51332563/00000...
000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,/data8/jpg/files/p10/p10934092/s55245226/00004...
00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,/data8/jpg/files/p14/p14851532/s59116935/00005...
00005944-56375fa0-8329f4ae-e3dee4c3-4c4bc572,/data8/jpg/files/p13/p13424439/s53667003/00005...
00009bca-0893e5e4-c1296676-181f5233-25dfa9a3,/data8/jpg/files/p15/p15342241/s58246830/00009...
0000c2f5-f02f9f3c-1ed14642-958de0ad-d6ce4d20,/data8/jpg/files/p17/p17744443/s56580856/0000c...
0000d3be-591ae3b7-b03a7497-8319c02b-650bb4ab,/data8/jpg/files/p19/p19298916/s50189753/0000d...
0000e00d-372d678f-659e32be-5b82fe32-0d73ef1a,/data8/jpg/files/p15/p15468322/s59674182/0000e...
00010785-aefa6d86-8d00afab-09e76c6a-edb6b458,/data8/jpg/files/p18/p18778431/s59340602/00010...


In [362]:
meta_data = (pd.read_csv(META_DATA,
                            usecols=['dicom_id',
                                     'subject_id',
                                     'study_id',
                                     'ViewPosition',
                                     'PatientOrientation']
                            )[['dicom_id',
                               'subject_id',
                               'study_id',
                               'ViewPosition',
                               'PatientOrientation']]).set_index('dicom_id').sort_values(by='dicom_id')

meta_data['subject_study_id'] = meta_data['subject_id'] * 100_000_000 + meta_data['study_id']

meta_data = meta_data[(meta_data.ViewPosition=='AP') & (meta_data.PatientOrientation=='Erect')]
meta_data = meta_data.drop(columns=['subject_id', 'study_id', 'ViewPosition', 'PatientOrientation'])
print(f"meta_data shape: {meta_data.shape}")
meta_data.head(10)

meta_data shape: (116390, 1)


,subject_study_id
dicom_id,
000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,1093409255245226
00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,1485153259116935
00030f67-6141afb8-618a939e-80b252b4-c7d59aec,1538842154221476
00036658-c5362a79-400c2649-efb1304e-53c92b83,1827601050461256
00038f14-86928b2a-f3e34fd7-d44ecdff-5b8660a8,1770854550391163
00046130-fd952ef0-57f2948d-491a16b4-5db3a18c,1518483659382057
00046bce-20d53086-5ffea7f9-66324915-834d4778,1525512051300200
000512db-61f1b4e0-5b16d40e-f6aeb922-21f71de8,1595802455562738
00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,1062755655547878


In [364]:
labels = pd.read_csv(LABEL_PATH)
labels['subject_study_id'] = labels['subject_id'] * 100_000_000 + labels['study_id']
labels = labels.drop(columns=['subject_id', 'study_id'])

choose_column = 'Pleural Effusion'
findings = ['Atelectasis','Cardiomegaly','Consolidation', 'Edema', 'Enlarged Cardiomediastinum','Fracture',
             'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other','Pneumonia',
             'Pneumothorax', 'Support Devices']
     
labels = labels.drop(columns=[f for f in findings if f not in choose_column])
labels['Pleural Effusion'] = [1 if lbl==1.0 else 0 for lbl in pe]
labels = labels.set_index('subject_study_id')

print(f"Shape of labels: {labels.shape}")
labels.head(20)

Shape of labels: (227827, 1)


,Pleural Effusion
subject_study_id,
1000003250414267,0
1000003253189527,0
1000003253911762,0
1000003256699142,0
1000076457375967,0
1000089850771383,0
1000089854205396,0
1000093550578979,1
1000093551178377,0


df_label_data = pd.read_csv(LABEL_PATH)
df_label_data = df_label_data.drop(columns=[3], axis=1)

In [365]:
# meta_data = (jpg_paths.join(meta_data, on='dicom_id')).set_index(['subject_study_id'])
data_and_labels = meta_data.join(labels, on='subject_study_id')
print(f"Shape of data_and_labels: {data_and_labels.shape}")
data_and_labels.head(20)

Shape of data_and_labels: (116390, 2)


,subject_study_id,Pleural Effusion
dicom_id,,
000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d,1093409255245226,1.0
00005197-869d72f3-66210bf4-fa2c9d83-b613c4e7,1485153259116935,0.0
00030f67-6141afb8-618a939e-80b252b4-c7d59aec,1538842154221476,1.0
00036658-c5362a79-400c2649-efb1304e-53c92b83,1827601050461256,0.0
00038f14-86928b2a-f3e34fd7-d44ecdff-5b8660a8,1770854550391163,0.0
00046130-fd952ef0-57f2948d-491a16b4-5db3a18c,1518483659382057,0.0
00046bce-20d53086-5ffea7f9-66324915-834d4778,1525512051300200,0.0
000512db-61f1b4e0-5b16d40e-f6aeb922-21f71de8,1595802455562738,0.0
00054ff1-47f5c5c2-e8dc7c52-1deafa05-e1174cd0,1062755655547878,0.0


In [366]:
data_and_labels = pd.join(df1, df_label_data, how='left')

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False